

# Không Song Song



In [25]:
import numpy as np
import pandas as pd 
from collections import deque

In [26]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [27]:
df_raw = pd.read_csv('/content/gdrive/MyDrive/Bigdata/data/combined_data_1.txt', header=None, names=['User', 'Rating', 'Date'], usecols=[0, 1, 2])
df_raw.head()

,User,Rating,Date
0,1:,NaN,NaN
1,1488844,3.0,2005-09-06
2,822109,5.0,2005-05-13
3,885013,4.0,2005-10-19
4,30878,4.0,2005-12-26


Data file này chứa theo kiểu dòng đầu là Id Movie, ví dụ: "4500:" là id movie nên 2 cột còn lại trống, sau đó xuống dòng mới là Id User, Rating và Date, thể hiện 1 movie có những đánh giá của các User theo ngày.

In [28]:
print('Len of df_raw: ', len(df_raw))

Len of df_raw:  24058263


In [29]:
# Tìm hàng trống để phân chia dữ liệu cho mỗi phim
tmp_movies = df_raw[df_raw['Rating'].isna()]['User'].reset_index()
print(tmp_movies[:5])

   index User
0      0   1:
1    548   2:
2    694   3:
3   2707   4:
4   2850   5:


Ở bước này chủ yếu tìm cái số thứ tự dòng mà có Id Movie để dễ xử lý

In [30]:
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]
print(movie_indices[:5])

[[0, 1], [548, 2], [694, 3], [2707, 4], [2850, 5]]


Này chỉ ghép Id Movie với số thứ tự dòng của nó thôi

In [31]:
#  Chuyển điểm đầu và cuối của của tất cả các phim
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)
print(shifted_movie_indices)

deque([[548, 2], [694, 3], [2707, 4], [2850, 5], [3991, 6], [5011, 7], [5105, 8], [20016, 9], [20112, 10], [20362, 11], [20561, 12], [21108, 13], [21234, 14], [21353, 15], [21644, 16], [24344, 17], [31453, 18], [42176, 19], [42716, 20], [42833, 21], [43052, 22], [43256, 23], [43872, 24], [45206, 25], [46414, 26], [52276, 27], [52550, 28], [92303, 29], [92827, 30], [211241, 31], [211463, 32], [213318, 33], [220209, 34], [220318, 35], [221158, 36], [222098, 37], [222380, 38], [223183, 39], [223936, 40], [224428, 41], [224522, 42], [224651, 43], [224757, 44], [233259, 45], [235879, 46], [242438, 47], [244789, 48], [248381, 49], [248613, 50], [248942, 51], [249033, 52], [254181, 53], [254296, 54], [254456, 55], [256863, 56], [258697, 57], [262260, 58], [279666, 59], [279871, 60], [280167, 61], [280315, 62], [280606, 63], [280791, 64], [280887, 65], [281146, 66], [281313, 67], [281603, 68], [283820, 69], [283937, 70], [284281, 71], [285843, 72], [286023, 73], [286771, 74], [287160, 75], [28

deque() là hàng đợi hai đầu cho phép chúng ta thêm và xóa các phần tử ở cả hai đầu.

In [32]:
#Tạo biến để chứa dữ liệu đánh giá phim
user_data = []

# Cho vòng lặp chạy qua movie_indices và shifted_movie_indices
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Kiểm tra từng Id dòng trong biến nếu df_id_1<df_id_2 tức là những đánh giá của phim đó sẽ
    # nằm trong khoảng (df_id_1, df_id_2) nên sử dụng hàm loc để lọc ra các dòng đánh giá của 
    # phim đó và để vào biến tạm tmp_df
    # Nếu ngược lại tức là từ dòng df_id_1 sẽ là phim cuối trong data nên sẽ lọc dữ liệu từ 
    # dòng df_id_1 + 1 đến hết.
    if df_id_1<df_id_2:
        tmp_df = df_raw.iloc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = df_raw.iloc[df_id_1+1:].copy()
        
    # Tạo cột tên Movie_Id cho những dòng phim đó
    tmp_df['Movie_Id'] = movie_id
    
    # Thêm vào biến
    user_data.append(tmp_df)

# Xoá những biến này để giải phóng bộ nhớ
del df_raw, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
# Sau khi đã có data hoàn chỉnh thì đưa lại vào Dataframe
movie_info = pd.concat(user_data)
del user_data
print('Shape User-Ratings:\t{}'.format(movie_info.shape))
movie_info.sample(5)

Shape User-Ratings:	(24049266, 4)


,User,Rating,Date,Movie_Id
6037783,765132,4.0,2005-06-16,1180
7135060,2404300,4.0,2004-04-06,1428
20607787,103040,3.0,2004-07-14,3905
9367411,1226957,1.0,2005-04-16,1844
4822964,1975711,3.0,2004-10-26,963


In [33]:
# Đây là dữ liệu để train
data_model = movie_info.drop(columns=['Date'])
data_model.head()

,User,Rating,Movie_Id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1


In [34]:
data_model.dtypes

User         object
Rating      float64
Movie_Id      int64
dtype: object

In [35]:
data_model['User']=data_model['User'].astype(int)
data_model.dtypes

User          int64
Rating      float64
Movie_Id      int64
dtype: object

In [36]:
columns_titles = ["Movie_Id","User","Rating"]
data_model=data_model.reindex(columns=columns_titles)
data_model.head()

,Movie_Id,User,Rating
1,1,1488844,3.0
2,1,822109,5.0
3,1,885013,4.0
4,1,30878,4.0
5,1,823519,3.0


In [37]:
Y_data = data_model.to_numpy()
print(Y_data)

[[1.000000e+00 1.488844e+06 3.000000e+00]
 [1.000000e+00 8.221090e+05 5.000000e+00]
 [1.000000e+00 8.850130e+05 4.000000e+00]
 ...
 [4.499000e+03 5.125360e+05 5.000000e+00]
 [4.499000e+03 9.889630e+05 3.000000e+00]
 [4.499000e+03 1.704416e+06 3.000000e+00]]


In [38]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class CF(object):
    def __init__(self, Y_data, k, dist_func = cosine_similarity):
        self.Y_data = Y_data
        self.k = k # số lấy điểm neighbors
        self.dist_func = dist_func
        self.Ybar_data = None
        # số lượng user và item
        self.n_users = int(np.max(self.Y_data[:, 1])) + 1
        self.n_movies = int(np.max(self.Y_data[:, 0])) + 1
    def normalize_Y(self):
        movies = self.Y_data[:, 0] # tất cả movies trong data
        self.Ybar_data = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in range(1, self.n_movies):
            ids = np.where(movies == n)[0].astype(np.int32)
            user_ids = Y_data[ids, 1]
            # các xếp hạng tương ứng của movie
            ratings = Y_data[ids, 2]
            # lấy trung bình của các xếp hạng
            m = np.mean(ratings)
            if np.isnan(m):
              # để tránh trường hợp null
                m = 0 
            self.mu[n] = m
            # chuẩn hoá
            self.Ybar_data[ids, 2] = ratings - self.mu[n]

        self.Ybar = sparse.coo_matrix((self.Ybar_data[:, 2],
            (self.Ybar_data[:, 1], self.Ybar_data[:, 0])), (self.n_users, self.n_movies))
        self.Ybar = self.Ybar.tocsr()
        print(self.Ybar)

    def similarity(self):
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)
    def refresh(self):
        self.normalize_Y()
        self.similarity() 
        
    def fit(self):
        self.refresh()

    def __pred(self, m, u, normalized = 1):
        # tìm tất cả movie đã được đánh giá
        ids = np.where(self.Y_data[:, 1] == u)[0].astype(np.int32)
        # những bộ phim đã được đánh giá bởi người dùng
        movies_rated_by_u = (self.Y_data[ids, 0]).astype(np.int32)
        # tìm điểm tương đồng giữa movie hiện hành với những movie khác
        sim = self.S[m, movies_rated_by_u]
        # Tìm k movie có lượt đánh gía cao nhất
        a = np.argsort(sim)[-self.k:] 
        # mức độ phù hợp (tương tự)
        nearest_s = sim[a]
        # Mỗi người dùng 'near' đã xếp hạng mục i cho phim nào
        r = self.Ybar[u, movies_rated_by_u[a]]
        if normalized:
            # 1e-8 là đối số được thêm vào để tránh trường hợp bằng 0
            return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8)

        return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8) + self.mu[m]
    
    def pred(self, m, u, normalized = 1):
        return self.__pred(m, u, normalize)
    def recommend(self, m, normalized = 1):
        ids = np.where(self.Y_data[:, 0] == m)[0]
        users_interest_movies = self.Y_data[ids, 1].tolist()              
        recommended_users = []
        for i in range(self.n_users):
            if i not in users_interest_movies:
                rating = self.__pred(m, i)
                if rating > 0:
                    recommended_users.append(i)
                    if len(recommended_users) == 10: break
        
        return recommended_users

In [39]:
import time

start = time.time()
rs = CF(Y_data, k = 30)
rs.fit()

recommended_users = rs.recommend(2755)
end = time.time()

print("Run time: ", end-start)

print("Recomend movie ", 2775," for users:")
print(recommended_users)

  (6, 30)	-0.761831571124548
  (6, 157)	-0.37784090909090917
  (6, 173)	0.4608501118568231
  (6, 175)	0.9961525279439245
  (6, 191)	-1.9322015012307663
  (6, 197)	-0.4789377176681966
  (6, 241)	-1.1579276918438808
  (6, 295)	0.6853521224052774
  (6, 299)	-0.7066529768789884
  (6, 329)	0.41216328378480327
  (6, 361)	-0.6358234769024773
  (6, 445)	-0.5394928979158369
  (6, 457)	1.1256070091897121
  (6, 468)	-0.5170490283709075
  (6, 494)	-1.001485507246377
  (6, 501)	-0.8006324943997893
  (6, 528)	1.0168699258426175
  (6, 564)	0.8042313377679231
  (6, 580)	-0.07550822846079397
  (6, 658)	-0.6329090909090911
  (6, 705)	-0.6400889371493577
  (6, 706)	-0.53003003003003
  (6, 723)	0.0056279831872907415
  (6, 788)	-0.7645661757167757
  (6, 825)	-1.1093510475217165
  :	:
  (2649429, 2755)	1.3139448173005221
  (2649429, 2782)	0.7055752212389379
  (2649429, 2862)	0.688317957102262
  (2649429, 3106)	1.3507642324448055
  (2649429, 3174)	2.242237723977568
  (2649429, 3254)	1.0203745988152075
  (264

# Song Song Hoá

In [40]:
!pip install pyspark

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [42]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [43]:
data_split = train_test_split(Y_data,test_size=0.05)
data_train=data_split[1]
data_train

array([[3.624000e+03, 8.101680e+05, 5.000000e+00],
       [2.525000e+03, 7.986530e+05, 4.000000e+00],
       [2.102000e+03, 2.178481e+06, 5.000000e+00],
       ...,
       [8.310000e+02, 2.445180e+06, 4.000000e+00],
       [3.894000e+03, 1.342568e+06, 5.000000e+00],
       [3.427000e+03, 4.093430e+05, 2.000000e+00]])

In [44]:
rdd = spark.sparkContext.parallelize(data_train)

In [45]:
class CF(object):
    def __init__(self, rdd, k, dist_func = cosine_similarity):
        self.rdd = rdd
        self.k = k # số lấy điểm neighbors
        self.dist_func = dist_func
        # number of users and items
        self.n_users = int(self.rdd.map(lambda x: x[1]).max()) + 1
        self.n_movies = int(self.rdd.map(lambda x: x[0]).max()) + 1
    def normalize_Y(self):
        means = self.rdd.map(lambda x: (x[0], (x[2], 1))).reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])).map(lambda x: (x[0], x[1][0]/x[1][1]))
        self.normalize_data = self.rdd.map(lambda x: (x[0], (x[1], x[2]))).leftOuterJoin(means).map(lambda x: [x[0], x[1][0][0], x[1][0][1] - x[1][1]]).collect()
        self.normalize_data = np.array(self.normalize_data)
        
        self.Ybar = sparse.coo_matrix((self.normalize_data[:, 2],
            (self.normalize_data[:, 1], self.normalize_data[:, 0])), (self.n_users, self.n_movies))
        self.Ybar = self.Ybar.tocsr()
        print(self.Ybar)

    def similarity(self):
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)
    def refresh(self):
        """
        Normalize data and calculate similarity matrix again (after
        some few ratings added)
        """
        self.normalize_Y()
        self.similarity() 
        
    def fit(self):
        self.refresh()

    def __pred(self, m, u, normalized = 1):
        movies_rated_by_u = np.array(self.rdd.filter(lambda x: x[1] == u).map(lambda x: x[0]).collect()).astype(np.int32)
        
        sim = self.S[m, movies_rated_by_u]

        a = np.argsort(sim)[-self.k:] 

        nearest_s = sim[a]

        r = self.Ybar[u, movies_rated_by_u[a]]
        return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8)
    
    def pred(self, m, u, normalized = 1):
        return self.__pred(m, u, normalize)
    def recommend(self, m, normalized = 1):
        users_not_interest_movie = self.rdd.filter(lambda x: x[0] != m).map(lambda x: x[1]).collect()              
        recommended_users = []
        for i in users_not_interest_movie:
            rating = self.__pred(m, i)
            if rating > 0:
                recommended_users.append(i)
                if len(recommended_users) == 10: break
        
        return recommended_users

In [46]:
import time

start = time.time()
rs = CF(rdd, k = 30)
rs.fit()

recommended_users = rs.recommend(2755)
end = time.time()

print("Run time: ", end-start)

print("Recomend movie ", 2775," for users:")
print(recommended_users)

  (6, 30)	-0.745791532052372
  (6, 705)	-0.6477857878475799
  (6, 1096)	-0.24038461538461542
  (6, 3290)	0.6004521051144387
  (6, 3925)	-0.5218437615697891
  (7, 1220)	0.9902955447728274
  (7, 2122)	1.3335228349440968
  (7, 2452)	0.5583804143126176
  (7, 2699)	0.547945205479452
  (7, 2874)	0.45339990489776527
  (7, 3624)	1.0729001584786055
  (7, 4266)	1.2200374531835205
  (7, 4353)	0.6428571428571432
  (8, 1799)	1.3437009110901665
  (42, 2457)	0.4216799091940975
  (42, 3151)	-0.4000697593303104
  (42, 3371)	0.07562349155269521
  (59, 758)	1.480695718654434
  (59, 886)	0.8223234624145785
  (59, 1092)	-0.3017241379310347
  (59, 1245)	-1.193050193050193
  (59, 1470)	-0.7995149555375911
  (59, 2391)	-1.2891252006420544
  (79, 962)	-0.07096171802054174
  (79, 1703)	0.16638795986622057
  :	:
  (2649337, 1590)	1.0465549348230914
  (2649351, 2152)	0.5720232088799193
  (2649370, 3197)	0.7943573667711599
  (2649375, 3427)	0.6758539803547867
  (2649376, 2023)	-0.014705882352941124
  (2649378, 457